### Build a DNN using Keras with `RELU` and `ADAM`

#### Load tensorflow

In [1]:
import keras

Using TensorFlow backend.


#### Collect Fashion mnist data from tf.keras.datasets 

In [2]:
from keras.datasets import fashion_mnist

#### Change train and test labels into one-hot vectors

In [3]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

from keras.utils import np_utils

y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

#### Build the Graph

#### Initialize model, reshape & normalize data

In [4]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

In [5]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1).astype('float32')
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1).astype('float32')

In [6]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D
from keras.layers import Dense, Activation, Dropout, Flatten, Reshape

# Define model
model = Sequential()

#### Add two fully connected layers with 200 and 100 neurons respectively with `relu` activations. Add a dropout layer with `p=0.25`

In [7]:
# 1st Conv Layer
model.add(Conv2D(32, (3, 3), input_shape=(28, 28, 1)))
model.add(Activation('relu'))

# 2nd Conv Layer
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))

# Fully Connected Layer
model.add(Flatten())
model.add(Dense(200))
model.add(Dense(100))
model.add(Activation('relu'))

### Add the output layer with a fully connected layer with 10 neurons with `softmax` activation. Use `categorical_crossentropy` loss and `adam` optimizer and train the network. And, report the final validation.

In [18]:
# Output Layer
model.add(Dense(10))
model.add(Activation('softmax'))

# Loss and Optimizer
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
# Store Training Results
early_stopping = keras.callbacks.EarlyStopping(monitor='val_acc', patience=5, verbose=1, mode='auto')
callback_list = [early_stopping]

# Train the model
model.fit(x_train, y_train, batch_size=32, epochs=10,
           validation_data=(x_test, y_test), callbacks=callback_list)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 252s 4ms/step - loss: 0.3777 - acc: 0.8629 - val_loss: 0.3055 - val_acc: 0.8936
Epoch 2/10
60000/60000 [==============================] - 244s 4ms/step - loss: 0.2390 - acc: 0.9113 - val_loss: 0.2609 - val_acc: 0.9054
Epoch 3/10
60000/60000 [==============================] - 242s 4ms/step - loss: 0.1825 - acc: 0.9319 - val_loss: 0.2562 - val_acc: 0.9098
Epoch 4/10
60000/60000 [==============================] - 236s 4ms/step - loss: 0.1375 - acc: 0.9489 - val_loss: 0.2822 - val_acc: 0.9116
Epoch 5/10
60000/60000 [==============================] - 232s 4ms/step - loss: 0.1013 - acc: 0.9626 - val_loss: 0.3657 - val_acc: 0.9061
Epoch 6/10
60000/60000 [==============================] - 232s 4ms/step - loss: 0.0790 - acc: 0.9705 - val_loss: 0.3397 - val_acc: 0.9105
Epoch 7/10
60000/60000 [==============================] - 234s 4ms/step - loss: 0.0619 - acc: 0.9771 - val_loss: 0.4160 - 

### The final validation with test data gives 90% accuracy 

## Word Embeddings in Python with Gensim

In this, you will practice how to train and load word embedding models for natural language processing applications in Python using Gensim.


1. How to train your own word2vec word embedding model on text data.
2. How to visualize a trained word embedding model using Principal Component Analysis.
3. How to load pre-trained word2vec word embedding models.

### Run the below two commands to install gensim and the wiki dataset

In [8]:
#!pip install --upgrade gensim --user

In [9]:
#!pip install wikipedia --user

### Import gensim

In [10]:
import gensim

### Obtain Text

Import search and page functions from wikipedia module
search(/key word/): search function takes keyword as argument and gives top 10 article titles matching the given keyword.

page(/title of article/): page function takes page title as argument and gives content in the output.

In [15]:
## Usage: 

#from wikipedia import search, page
#titles = search("Machine Learning")
#wikipage = page(titles[0])
#print(wikipage.content)

### Print the top 10 titles for the keyword `Machine Learning`

In [16]:
from wikipedia import search, page
titles = search("Machine Learning")
print(len(titles))
for title in titles:
    print(title)

10
Machine learning
Active learning (machine learning)
Boosting (machine learning)
List of datasets for machine learning research
Deep learning
Support-vector machine
Outline of machine learning
Adversarial machine learning
Weka (machine learning)
Learning


### Get the content from the first title from the above obtained 10 titles.

In [17]:
wikipage = page(titles[0])
print(wikipage.content)

Machine learning (ML) is the scientific study of algorithms and statistical models that computer systems use to effectively perform a specific task without using explicit instructions, relying on patterns and inference instead. It is seen as a subset of artificial intelligence. Machine learning algorithms build a mathematical model of sample data, known as "training data", in order to make predictions or decisions without being explicitly programmed to perform the task. Machine learning algorithms are used in the applications of email filtering, detection of network intruders, and computer vision, where it is infeasible to develop an algorithm of specific instructions for performing the task. Machine learning is closely related to computational statistics, which focuses on making predictions using computers. The study of mathematical optimization delivers methods, theory and application domains to the field of machine learning. Data mining is a field of study within machine learning, a

### Create a list with name `documents` and append all the words in the 10 pages' content using the above 10 titles.

In [28]:
import re, string

def clean_str(string):
  """
  String cleaning before vectorization
  """
  try:    
    string = re.sub(r'^https?:\/\/<>.*[\r\n]*', '', string, flags=re.MULTILINE)
    string = re.sub(r"[^A-Za-z]", " ", string)         
    words = string.strip().lower().split()    
    words = [w for w in words if len(w)>=1]
    return " ".join(words)	
  except:
    return ""

In [30]:
documents = []

for title in titles:
    clean_doc = clean_str(page(title).content)
    documents.append(clean_doc.split(' '))

print(len(documents))
print(documents[0])

10
['machine', 'learning', 'ml', 'is', 'the', 'scientific', 'study', 'of', 'algorithms', 'and', 'statistical', 'models', 'that', 'computer', 'systems', 'use', 'to', 'effectively', 'perform', 'a', 'specific', 'task', 'without', 'using', 'explicit', 'instructions', 'relying', 'on', 'patterns', 'and', 'inference', 'instead', 'it', 'is', 'seen', 'as', 'a', 'subset', 'of', 'artificial', 'intelligence', 'machine', 'learning', 'algorithms', 'build', 'a', 'mathematical', 'model', 'of', 'sample', 'data', 'known', 'as', 'training', 'data', 'in', 'order', 'to', 'make', 'predictions', 'or', 'decisions', 'without', 'being', 'explicitly', 'programmed', 'to', 'perform', 'the', 'task', 'machine', 'learning', 'algorithms', 'are', 'used', 'in', 'the', 'applications', 'of', 'email', 'filtering', 'detection', 'of', 'network', 'intruders', 'and', 'computer', 'vision', 'where', 'it', 'is', 'infeasible', 'to', 'develop', 'an', 'algorithm', 'of', 'specific', 'instructions', 'for', 'performing', 'the', 'task',

### Build the gensim model for word2vec with by considering all the words with frequency >=1 with embedding size=50

In [31]:
#Build the model
model = gensim.models.Word2Vec(documents, #Word list
                               min_count=1, #Ignore all words with total frequency lower than this                           
                               workers=4, #Number of CPUs
                               size=50,  #Embedding size
                               window=5, #Maximum Distance between current and predicted word
                               iter=10   #Number of iterations over the text corpus
                              )  

### Exploring the model

#### Check how many words in the model

In [33]:
print('Number of words:',model.wv.syn0.shape[0])

Number of words: 4996


C:\AIML\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  """Entry point for launching an IPython kernel.


### Get an embedding for word `SVM`

In [35]:
model.wv['svm']

array([ 0.2695292 , -0.07491785,  0.22683716, -0.8812913 , -0.04476273,
        0.8587758 , -0.5732648 , -0.6432545 ,  0.09135519,  0.49277425,
        0.8205995 ,  0.9822047 ,  0.41738817, -1.251793  , -0.42116922,
       -0.66463006,  0.5399261 ,  0.71219796,  0.13837677, -0.9004556 ,
        0.7570285 , -0.6750986 , -0.62499577,  0.42589387,  0.02156076,
       -0.08103542,  0.5092484 ,  0.59049386, -0.43134278,  1.0235611 ,
       -0.29856876,  1.0840778 ,  0.66862845,  0.33343843,  0.5420241 ,
       -0.8272531 , -0.164085  ,  1.2281519 , -2.0486572 , -0.9192366 ,
       -0.4527347 , -0.24418233, -1.6572928 ,  0.20031801,  0.69819236,
       -0.40072313,  0.10640454, -0.7934537 ,  0.23233157, -0.72280496],
      dtype=float32)

### Finding most similar words for word `learning`

In [36]:
model.wv.most_similar('learning')

[('machine', 0.9976972937583923),
 ('networks', 0.9962754249572754),
 ('deep', 0.996139407157898),
 ('supervised', 0.9956562519073486),
 ('neural', 0.9956175684928894),
 ('algorithms', 0.9950774908065796),
 ('belief', 0.992693305015564),
 ('build', 0.9909639358520508),
 ('focuses', 0.9900330901145935),
 ('part', 0.9884830713272095)]

### Find the word which is not like others from `machine, svm, ball, learning`

In [40]:
words = list(['machine', 'svm', 'ball', 'learning'])
model.wv.doesnt_match(words)

'ball'

### Save the model with name `word2vec-wiki-10`

In [41]:
model.save('word2vec-wiki-10')

### Load the model `word2vec-wiki-10`

In [43]:
model = gensim.models.Word2Vec.load('word2vec-wiki-10')

In [45]:
#Testing the loaded model
words = list(['regression', 'classification', 'support', 'algorithm'])
model.wv.doesnt_match(words)

'support'